### Import necessary modules

In [1]:
import os
import zipfile
import pandas as pd
from IPython.display import display

### Important constants (expected from user)

In [2]:
DATA_DIR = "firefox_data"

### Unpack archives and get tables

In [3]:
FILES_DIR = f"{DATA_DIR}_files"

In [4]:
DFS = dict()
for archive in os.listdir(DATA_DIR):
    path = os.path.join(DATA_DIR, archive)
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(FILES_DIR)
    DFS[archive.split('_')[0]] = pd.read_csv(f"{os.path.join(FILES_DIR, archive)[:-3]}csv")

### Define utility functions

In [31]:
# Key is usually OS
# Function is able to work with the beginning of a key
def head(
    key: str, number_of_rows: int=5,
    show: bool=False
):
    for df_key in DFS:
        if df_key.startswith(key):
            df_head = DFS[df_key].head(number_of_rows)
            if show:
                display(df_head)
            return df_head

In [45]:
# Function is able to work with the beginning of a key
# If keys == None, all keys will be used
def find_instruction(
    instruction: str,
    keys: None|list[str]=None,
    only_filenames: bool=True,
    show: bool=False,
    number_of_rows_to_show: int=5
):
    if keys is None:
        keys = list(DFS)
    tables = dict()
    for key in keys:
        for df_key in DFS:
            if df_key.startswith(key):
                try:
                    table = DFS[df_key][DFS[df_key][instruction] != 0]
                    if only_filenames:
                        table = list(table.filename)
                    tables[df_key] = table
                except KeyError:
                    tables[df_key] = None
    if show:
        for key in tables:
            print(f"{key}:")
            if tables[key] is None:
                continue
            if only_filenames:
                for filename in tables[key]:
                    print(f"\t{filename}")
            else:
                display(tables[key].head(number_of_rows_to_show))
            print()
            
    return tables